**AGENT-BASED ARTIFICIAL INTELLIGENCE, POLIBA, A.Y. 2023/2024**

**LOCAL SEARCH**

List of Contributors:
- Tommaso Di Noia
- Alberto Carlo Maria Mancino
- Vincenzo Paparella

### Introduction to Local Search

So far, we have studied how to solve search problems with uninformed and informed strategies. In both cases, our goal was to find a path, i.e., a sequence of actions, through the search space to reach a goal state.

However, sometimes we care only about the final state, not the path to get there. For example, in the 8-queens problem, we care only about finding a valid final configuration of 8 queens such that the queens are not in conflict among them. Indeed, it is trivial to reconstruct the steps that created the valid configuration.

In this context, **local search** algorithms operate by searching from an initial state to neighboring states solely, without keeping track of the paths, nor the set of states that have been reached.

Practically, within local search algorithms, we design a **value function** such that the closer we are to the goal state, the higher is the value function. In this way, the maximum value of this function is associated to the goal state.

### 8-queens Problem

To illustrate the local search algorithms, we will utilize the 8-queens problem as a toy-example.

In this problem, we must place 8 queens on a 8x8 chess board so that no queen attacks another. A queen attacks any piece in the same row, column, or diagonal.

Consequently, the goal state is a state in which the queens are placed such that the number of conflicts is 0.

The initial state can be self-declared or chosen at random.

Ok, let's start!

#### Node's implementation

In local search algorithms, given a state, we move into one of its neighbors. Then, we move along a tree, as we did in uninformed and informed search strategies. However, we are not interested in the path along the tree.

Given this observation, we will reuse the node's implementation seen for the tree search strategy. If you don't have familiarity with the following class, you should study tree and graph search first.

In [1]:
class Node:
    def __init__(self, state, parent, action, cost, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.depth = depth

    def __repr__(self):
        """
        A representation of the class. Useful with functions like print.
        :return: a string
        """
        return f'({self.state})'

    def expand(self, state, action, cost=1):
        """
        Given a new state returns a child tree node containing that state
        :param new_state: state that will be contained by the node
        :param action: action that led to the state
        :param cost: cost of the path of the previous node
        :return: a child node
        """
        return Node(state=state,
                    parent=self,
                    action=action,
                    cost=self.cost+cost,
                    depth=self.depth+1)

    def path(self):
        """
         Returns the path from the root node to the actual node
        :return: a list of actions
        """
        path = []
        node = self
        while node.parent:
            path.append(node.action)
            node = node.parent
        path = list(reversed(path))
        return path


#### Modeling the environment

Firstly, we should model the chess board in such a way our machine can understand it.

We can suppose to have one queen per column on our board. In this way, we already solve the conflicts among the queens due to the same column. In addition, this choice makes modeling the environment much easier.

Indeed, now we can model our chess board as a tuple of eight elements in which each element corresponds to a queen. Specifically, the value of the element indicates the row of the queen, while the element's index within the tuple indicates the column of the queen.

For instance, in the tuple (0,1,3,5,6,1,1) --- Please, observe that the maximum value of a row is 7 since the indexing starts from 0! --- :
- the queen in column 0 is in row 0;
- the queen in column 1 is in row 1;
- the queen in column 2 is in row 3;
- the queen in column 3 is in row 5;
- and so son.

However, as a good practice, we should not mix the concepts of environment and state. Actually, a populated tuple like the one in the previous example represents a state, i.e., a particular disposition of queens on the chess board.

If we imagine the empty chess board, i.e., the environment only, it is an empty tuple (or a tuple with eight None within it (None, None, None, None, None, None, None, None)). Obviously, we are not required to implement an empty tuple! Then, we have just conceptualized our environment.

#### Problem formulation

As we have seen in the tree and graph search, formulating the problem practically means modeling the agent. Therefore, we implement the EightQueensProblem Class in a similar way we have implemented the problem-related class for the tree/graph search:
* the _successors_ function remains the same. It returns the reachable states from a state with the respective actions;
* the _actions_ function, given a state, returns the list of possible actions. Here, the possible actions include moving each queen on a row of its associated column except for the row in which it stands on on the current state. In a list, we collect all the coordinates (col, row) of the chess board corresponding to the possible actions;
* the _return_ function, given a state and an action, returns the reached state by performing the action. Given that an action is represented by the couple (col, row) as said above, this function is responsible of updating the row "row" of the column "col" of the current state;
* the _cost_ function returns the cost of an action. In this problem, we do not have a policy for the cost. Then, it is always unitary;
* the _goal_test_ function checks if the goal condition has been reached. For the 8-queens problem, the goal state is a state in which no queens are in conflict among them.

Consequently, we implement the _conflicts_ function that, given a state, returns the number of conflicts among the queens. Then, given a queen, we have a conflict with a subsequent queen in three cases:
- queens on the same column. We have already solved this conflict by supposing to have a chess board with only one queen on each column;
- queens on the same row;
- queens on the same diagonal. We notice that the conflict along the diagonals can be described as follows. If the diagonal has a positive slope, the sum between the rows' and columns' indices of the diagonal is constant. Then, two queens are in conflict if the sum of their indices is the same. If the diagonal has a negative slope, the difference between the rows' and columns' indices of the diagonal is constant. Then, two queens are in conflict if the difference of their indices is the same.

Finally, we implement the _value_ function. We recall that the closer we are to the goal state, the higher is the value function. In this way, the maximum value of this function is associated to the goal state. For the 8-queens Problem, a valid value function would be the number of maximum conflicts minus the number of conflicts in the current state. In this way, if we are in the goal state, the function would return 0. The less is the number of conflicts (the closer we are to the goal state), the higher is the number returned.

In [2]:
class EightQueensProblem:

    def __init__(self, initial_state=None):
        if initial_state is None:
            initial_state = self.random()
        self.initial_state = initial_state
        self.max_conflicts = sum([i for i in range(1, 8)])

    def successors(self, state):
        """
        Given a state returns the reachable states with the respective actions
        :param state: actual state
        :return: list of successor states and actions
        """
        possible_actions = self.actions(state)
        return [(self.result(state, a), a) for a in possible_actions]

    def actions(self, state):
        """
        Given a state returns the list of possible actions
        :param state: actual state
        :return: a list of actions
        """
        actions = []
        for col, queen in enumerate(state):
            squares = list(range(0, 8))
            squares.remove(queen)
            # new_actions = list(zip(squares, [col]*len(squares)))
            new_actions = list(zip([col] * len(squares), squares))
            actions.extend(new_actions)
        return actions

    def result(self, state=None, action=None):
        """
        Given a state and an action returns the reached state
        :param state: actual state
        :param action: chosen action
        :return: reached state
        """
        new_state = list(state)
        col, new_row = action
        new_state[col] = new_row
        return tuple(new_state)

    def conflicts(self, state):
        """
        Given a state return the number of conflicts
        :param state: a state
        :return: number of conflicting queens
        """
        conflicts = 0
        for col in range(8):
            queen = state[col]
            for col1 in range(col+1, 8):
                queen1 = state[col1]
                if queen == queen1:
                    conflicts += 1
                if queen - col == queen1 - col1 or queen + col == queen1 + col1:
                    conflicts += 1
        return conflicts

    def goal_test(self, state):
        """
        Checks if the goal condition has been reached
        :param state: actual state
        :return: True if the goal condition is matched, False otherwise
        """
        return self.conflicts(state) == 0

    def cost(self, state, action):
        """
        Returns the cost of an action. In this problem the cost is always unitary.
        :param state: a state
        :param action: an action
        :return: a cost
        """
        return 1

    def value(self, state):
        """
        Returns the value of a state. This function is used for evaluating a state in the local search.
        (The higher the better)
        :param state: a state
        :return: the value of a state
        """
        return self.max_conflicts - self.conflicts(state)

    @staticmethod
    def random():
        """
        Generate a random chess with 8 queens
        :return: a tuple with 8 elements
        """
        chess = [random.randrange(0, 8) for _ in range(8)]
        return tuple(chess)

    @staticmethod
    def print_chess(state):
        print('\t', end='')
        for number in [1, 2, 3, 4, 5, 6, 7, 8]:
            print(f"|  {number}  ", end='')
        print('|', end='')
        print('\n\t_________________________________________________')

        for row, letter in zip(range(8), ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']):
            print(letter + '\t', end='')
            print('|', end='')

            for queen in state:
                if queen == row:
                    print('  Q  ', end='')
                else:
                    print('     ', end='')
                print('|', end='')
            print('\n', end='')
            print('\t_________________________________________________')


### Hill Climbing

The first local search algorithm is Hill Climbing. Given the _value_ function to provide a score associated to a state, this strategy keeps track of one current state. On each iteration, it moves to the neighboring state having the highest value. The algorithm ends when no neighboring state have higher values than the current one.

Therefore, the algorithm could be implemented in the HillClimbing Class receiving in input the problem. Then, in the _run_ function, we follow these steps:
1. We initialize a node with the initial state;

We loop the following steps:

2. we expand the current state by finding the neighbors with the _successors_ function;
3. we select the neighbor having the maximum value returned by the _value_ function;
4. we check if the value of the current state is the greater than the one of the selected neighbor. If so, we have a solution. Otherwise, we expand the selected neighbor.

The implementation of the HillClimbing Class is below.

In [12]:
class HillClimbing:

    def __init__(self, problem):
        self.problem = problem

    def __repr__(self):
        return 'Hill Climbing'

    def run(self):
        # initial node with initial state
        node = Node(state=self.problem.initial_state,
                    parent=None,
                    action=None,
                    cost=0,
                    depth=0)

        while True:
            # the local search looks only to the nearest states
            new_states = self.problem.successors(node.state)

            # no more neighbours to explore
            if not new_states:
                return 'Stop', node.state

            # select the best neighbour given the objective function
            best_neighbor, best_action = max(new_states, key=lambda x: self.problem.value(x[0]))

            # check if there is not any improvement
            if self.problem.value(node.state) >= self.problem.value(best_neighbor):
                return 'Ok', node.state

            # next neighbour to explore
            node = node.expand(state=best_neighbor,
                               action=best_action,
                               cost=1)


### Simulated Annealing

The Hill Climbing algorithm has a drawback. This algorithm can get stuck on local maximum and plateau of the value function. In these cases, the algorithm reaches a point at which no progress is being made. Then, a solution which is not the best solution is returned.

To overcome the limitations of Hill Climbing, the Simulated Annealing algorithm introduces a random component within the algorithm, by allowing some bad moves but gradually decreasing their size and frequency in time.

We follow these steps:
1. We initialize a node with the initial state;

We loop the following steps:

2. we expand the current state by finding the neighbors with the _successors_ function;
3. we select a random neighbor;
4. we check if the value of the neighbor state is the greater than the one of the current state (i.e., if their difference is positive). If so, we surely expand the neighbor. Otherwise, we decide to expand the neighbor with if a certain probability value is less than the score exp(-delta/temperature).

Here, delta is the value difference between the selected neighbor and the current state. Temp is the temperature factor. This temperature is updated during the loop as a function of the time (i.e., the number of iterations), so that its value decreases as the time increases. In the implementation below, there are three possible examples to update the temperature:
1. the time is decreased linearly according to a lambda factor;
2. the time is decreased exponentially by a factor exp(-lambda * time).

The iterations continue untile the temperature value is greater than a fixed threshold, tipically 0. Optionally, we can set a number of maximum iterations to be reached until we loop.

The implementation of the HillClimbing Class is below.

In [13]:
import math

class SimulatedAnnealing:

    def __init__(self, problem, min_temp=0, max_time=100, lam=0.001):
        self.problem = problem
        self.min_temp = min_temp
        self.max_time = max_time
        self.lam = lam

    def __repr__(self):
        return 'Simulated Annealing'

    def linear_schedule(self, temp):
        return temp - self.lam

    def exponential_schedule(self, temp, time):
        return temp * math.exp(-self.lam * time)

    def run(self, initial_temp=100):
        # set time at the beginning of the search
        time = 0
        temp = initial_temp

        # initial node with initial state
        node = Node(state=self.problem.initial_state,
                    parent=None,
                    action=None,
                    cost=0,
                    depth=0)

        while temp > self.min_temp and time < self.max_time:
            # the local search looks only to the nearest states
            new_states = self.problem.successors(node.state)

            # no more neighbours to explore
            if not new_states:
                return 'Stop', node

            # select the best neighbour given the objective function
            selected_neighbour, selected_action = random.choice(new_states)

            score_diff = self.problem.value(selected_neighbour) - self.problem.value(node.state)

            if score_diff > 0 or random.uniform(0, 1) < math.exp(score_diff / temp):
                # new state to explore
                node = node.expand(state=selected_neighbour,
                                   action=selected_action,
                                   cost=1)
            # update temperature
            temp = self.exponential_schedule(temp, time)
            # update time
            time += 1

        print(f'temp: {temp}, time: {time}')
        return 'Ok', node.state


### Genetic Algorithm

The Genetic Algorithm is a strategy for local search inspired by the metaphor of natural selection in biology.

There is a population of individuals (states), in which the fittest (highest value) individuals produce offspring (successor states) that populate the next generation.

In this kind of algorithm, a state must explicitly be described as a sequence of characters, like strings or tuples. We can observe that our problem fits this kind of requirement, as we modeled a state as a tuple of 8 numbers.

The algorithm works as follows:
1. we initially generate a random population. Practically, it means generating a certain number of random states.
2. upon this random population, we continue by generating a certain number of subsequent generations (composed by the current population).

The generations are created according to three steps. We perform the following steps a number of times equals to the population's size:
1. **selection**: we select two individuals (i.e., states) from the population in a random weighted way. "Random weighted way" means selecting the individuals in a random way. However, the individuals with a higher value score are more likely to be selected. We perform these operations in the _select_ function. This function returns two individuals from the population;
2. **crossover**: given the two selected individuals, we combine randomly them. Combining them randomly means that we merge their sequence of characters taking the first part of the sequence from the first individual, the second one from the second individual. The split index is generated randomly. We perform these operations in the _crossover_ function. This function returns an offspring (i.e., a state).
3. **mutation**: given the offspring from the crossover operations, we modify a gene of the offspring if a random probability is greater than a certain treshold. A gene is a character of the sequence that represents the state. Of course, the randomly modified gene must have an accepted value from the gene pool. The gene pool is the set of characters that a gene can assume. For instance, in the 8-queens problem, the accepted values is in the range 0,7 given the 8x8 chess board. Both the gene to modify and the value that it is going to assume are generated randomly.

Once a new generation is created, the population is updated. The operations described above are repeated for a fixed number of generations. Once we have finished generating generations, we return the state having the highest value function score, that will be the solution.

In [14]:
class Genetic:

    def __init__(self, problem, population=1000, generations=100, p_mutation=0.1, gene_pool=None):
        self.problem = problem
        self.population = population
        self.generations = generations
        self.couples = int(self.population / 2)
        self.p_mutation = p_mutation
        self.gene_pool = gene_pool

    def __repr__(self):
        return 'Genetic'

    def select(self, population):
        fitnesses = list(map(self.problem.value, population))
        return random.choices(population=population, weights=fitnesses, k=2)

    def crossover(self, couple):
        parent_a, parent_b = couple
        split = random.randrange(0, len(parent_a))
        return tuple(list(parent_a[:split]) + list(parent_b[split:]))

    def mutation(self, state):
        if random.uniform(0, 1) > self.p_mutation or self.gene_pool is None:
            return state
        new_state = list(state)
        new_state[random.randrange(len(state))] = random.choice(self.gene_pool)
        return tuple(new_state)

    def run(self):
        population = [self.problem.random() for _ in range(self.population)]
        for e in range(self.generations):
            best = max(population, key=lambda x: self.problem.value(x))
            print(f'Generation: {e} - max score: {self.problem.value(best)}')
            new_generation = [
                self.mutation(
                    self.crossover(
                        self.select(population)
                    )
                )
                for _ in range(self.population)]
            population = new_generation
        return 'ok', max(population, key=lambda x: self.problem.value(x))


### Test

Let's test the algorithms.

In [17]:
import random

# formulate the problem
problem = EightQueensProblem()

# search algorithm
search = HillClimbing(problem=problem)

# run algorithm
result, state = search.run()

# display the solutions
print(search)
print(result)
print(problem.value(state))
print(state)
problem.print_chess(state)

# search algorithm
search = SimulatedAnnealing(problem=problem, max_time=1000, lam=0.01)

# run algorithm
result, state = search.run()

# display the solutions
print(search)
print(result)
print(problem.value(state))
print(state)
problem.print_chess(state)

# search algorithm
search = Genetic(problem=problem, population=50, generations=100, p_mutation=0.1, gene_pool=list(range(8)))

# run algorithm
result, state = search.run()

# display the solutions
print(search)
print(result)
print(problem.value(state))
print(state)
problem.print_chess(state)

Hill Climbing
Ok
27
(6, 0, 3, 1, 5, 7, 2, 4)
	|  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |
	_________________________________________________
A	|     |  Q  |     |     |     |     |     |     |
	_________________________________________________
B	|     |     |     |  Q  |     |     |     |     |
	_________________________________________________
C	|     |     |     |     |     |     |  Q  |     |
	_________________________________________________
D	|     |     |  Q  |     |     |     |     |     |
	_________________________________________________
E	|     |     |     |     |     |     |     |  Q  |
	_________________________________________________
F	|     |     |     |     |  Q  |     |     |     |
	_________________________________________________
G	|  Q  |     |     |     |     |     |     |     |
	_________________________________________________
H	|     |     |     |     |     |  Q  |     |     |
	_________________________________________________
temp: 0.0, time: 388
Simulate